In [11]:
# A bot that will apply for jobs
url = "https://www.linkedin.com/jobs/search/?currentJobId=3509101579&keywords=marketing%20manager"

# url = "https://careers-peraton.icims.com/jobs/131931/software-engineer-ii/candidate?from=login&eem=MWnpnU8MV3MIeioSn89IA93rGjdGFiVZVpBictjqKHLiDJqX%252FfNbjubZV3kx5PlL&code=b7ed6762f06974c7bebe9d05fa1fe50fa33dfcc0b66bbd72e7df19cb72963012&ga=81a5c8777d484570e5f4030406aff2ae558d7345d3ee572bc4471d1023cec550&accept_gdpr=1"
# url = "https://www.tesla.com/careers/search/job/apply/159337?source=LinkedIn"

# url = "https://wd1.myworkdaysite.com/en-US/recruiting/fmr/FidelityCareers/job/Boston-MA/Principle-Software-Engineer---Quant-Developer_2071123/apply/"

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
import time
import datetime
import pickle
import atexit



# define a function to save cookies
def save_cookies(driver):
    print("Saving cookies")
    pickle.dump(driver.get_cookies(), open("cookies.pkl", "wb"))


driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()))
driver.get(url)

# load cookies from a file, if available
try:
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
except:
    pass


driver.get(url)
print("Here")


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 20.9MB/s]


Here


In [17]:
save_cookies(driver)

Saving cookies


In [8]:
corpus_file = "resume.json"
import json
import openai
import numpy as np


with open(corpus_file) as f:
    corpus = json.load(f)


def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
corpus_embeddings = {}

for key, value in corpus.items():
    corpus_embeddings[key] = get_embedding(key)

In [21]:
from time import sleep
driver.switch_to.window(driver.window_handles[-1])
# get url
url = driver.current_url
print(url)

i = 0
active_element = driver.switch_to.active_element

while True:
    sleep(1)
    new_element = driver.switch_to.active_element
    if new_element == active_element or (new_element.tag_name != "input" and new_element.tag_name != "textarea"):
        i += 1
        
        if(i % 10 == 0):
            driver.switch_to.window(driver.window_handles[-1])
        continue

    active_element = new_element

    
    # get element html
    search_query = active_element.get_attribute("outerHTML")
    print(str(search_query))

    
    search_query_embedding = get_embedding(search_query)

    # List matches sorted by score in descending order
    matches = sorted(
        corpus_embeddings.items(),
        key=lambda item: cosine_similarity(item[1], search_query_embedding),
        reverse=True,
    )

    # Get the match with the highest score
    top_match = matches[0]
    print("Top match: ", top_match[0])
    print("Score: ", cosine_similarity(top_match[1], search_query_embedding))

    if(cosine_similarity(top_match[1], search_query_embedding) < 0.786):
        print("No match found")
        try:
            name = active_element.get_attribute("name")
            # get label with active element name
            label = driver.find_element(By.XPATH, "//label[@for='" + name + "']")
            # get label text
            search_query = label.text
            print(str(search_query))
            search_query_embedding = get_embedding(search_query)

            matches = sorted(
                corpus_embeddings.items(),
                key=lambda item: cosine_similarity(item[1], search_query_embedding),
                reverse=True,
            )

            top_match = matches[0]
            print("Top match: ", top_match[0])
            print("Score: ", cosine_similarity(top_match[1], search_query_embedding))

            if(cosine_similarity(top_match[1], search_query_embedding) < 0.786):
                print("No match found")
                continue
        except Exception as e:
            print(e)
            continue


    # get the text
    text = corpus[top_match[0]]
    print(text)

    # send the text
    active_element.send_keys(text)




    



https://careers.united.com/us/en/apply/5/1?jobSeqNo=UAIUADUSWHQ00022003EXTERNALENUSTALEO
<input type="text" field-label="apply_field_label_city" name="workCity" id="workCity" autocomplete="off" value="" maxlength="100" class="form-control mandatory max100" placeholder="Enter City">
Top match:  city
Score:  0.8046568811847269
Austin
<input type="text" field-label="apply_field_label_school" name="institution" id="institution" autocomplete="off" value="" maxlength="100" class="form-control  clonedInputElm dyna-load mandatory max100" placeholder="Search Institution name">
Top match:  school
Score:  0.7871862395987576
University of Texas at Austin
<input type="text" field-label="apply_field_label_fieldofStudy" name="program" id="program" autocomplete="off" value="" maxlength="255" class="form-control clonedInputElm dyna-load max255" placeholder="Enter program name">
Top match:  school
Score:  0.769180334234622
No match found
Program
Top match:  school
Score:  0.845808250605527
University of

KeyboardInterrupt: 

In [24]:
# get all inputs textareas and select elements in order
for element in driver.find_elements(By.XPATH, "//input | //textarea | //select"):
    # get label
    try:
        name = element.get_attribute("name")
        # get label with active element name
        label = driver.find_element(By.XPATH, "//label[@for='" + name + "']")
        # get label text
        search_query = label.text

        # remove newline character
        search_query = search_query.replace("\n", " ")
        print(str(search_query) + ":")
    except:
        continue

Employer *
is a required field.:
Name during employment:
Employer City *
is a required field.:
Employer State *
is a required field.:
Job Title:
End Date *
is a required field.:
Supervisor's Name:
Supervisor's Phone:
Education *
is a required field.:
Country *
is a required field.:
Program:
Education Level *
is a required field.:
Start Date *
is a required field.:
Graduation Date:
Education *
is a required field.:
Country *
is a required field.:
Program:
Education Level *
is a required field.:
Start Date *
is a required field.:
Graduation Date:
